In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    base_url="",
    api_key = ""
)
def call_gpt4(messages):
  response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens=32,
        temperature=0,
        seed=0,
        messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
  )
  return response

# 测试调用
messages = "解释一下量子力学的基本原理。"
response = call_gpt4(messages)
print(response.model)

gpt-35-turbo


In [ ]:
import json
import re
import os
from tqdm import tqdm
import numpy as np
import random
from openai import OpenAI
client = OpenAI(
    base_url="",
    api_key = ""
)
# 设置模型和结果文件名
model = "gpt-3.5-turbo"
results_file = "reclor_test_gpt-3.5_zero_shot_results"

results_file_json = results_file + ".json"
if os.path.exists(results_file_json):
    with open(results_file_json, "r") as f:
        zero_shot_results = json.load(f)
else:
    zero_shot_results = []

# 读取数据文件
with open("/content/drive/MyDrive/project/test.json", "r") as f:
    data = json.load(f)

start = len(zero_shot_results)

for i in tqdm(range(start, len(data))):
    question = data[i]['context'] + " " + data[i]['question'] + \
        "\nA. " + data[i]['answers'][0] + \
        "\nB. " + data[i]['answers'][1] + \
        "\nC. " + data[i]['answers'][2] + \
        "\nD. " + data[i]['answers'][3] + \
        "\nAmong A to D, the answer is: "

    messages = [
        {"role": "user", "content": question},
    ]

    response = client.chat.completions.create(
        model=model,
        max_tokens=32,
        temperature=0,
        messages=messages)

    content = response.choices[0].message.content
    try:
        pred = re.findall(r'A|B|C|D', content)[0]
    except:
        try_time = 1
        while try_time < 5:
            response = client.chat.completions.create(
                model=model,
                max_tokens=32,
                temperature=1,
                messages=messages)
            content = response.choices[0].message.content
            pred = re.findall(r'A|B|C|D', content)
            if len(pred) > 0:
                pred = pred[0]
                break
            else:
                try_time += 1
                print(f"Try {try_time} times for {i}th question")

    results = json.loads(response.model_dump_json())
    results['pred'] = pred
    zero_shot_results.append(results)

    with open(results_file_json, "w") as f:
        json.dump(zero_shot_results, f, indent=4)

assert len(zero_shot_results) == len(data)
map_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
results_npy = []
for i in range(len(zero_shot_results)):
    results_npy.append(map_dict[zero_shot_results[i]['pred']])
print(results_npy)
print(len(results_npy))
results_npy = np.array(results_npy)
results_file_npy = results_file + ".npy"
np.save(results_file_npy, results_npy)

100%|██████████| 1000/1000 [17:48<00:00,  1.07s/it]

[3, 1, 2, 0, 1, 2, 1, 1, 3, 0, 3, 3, 3, 1, 2, 0, 3, 3, 2, 3, 1, 3, 0, 2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 1, 3, 2, 3, 2, 3, 2, 0, 3, 1, 3, 3, 3, 2, 2, 1, 0, 3, 1, 2, 3, 2, 0, 3, 3, 1, 0, 0, 1, 1, 3, 3, 1, 3, 3, 1, 1, 3, 2, 0, 3, 2, 2, 0, 1, 1, 0, 3, 1, 1, 2, 2, 2, 0, 0, 2, 2, 1, 3, 0, 0, 1, 2, 2, 2, 3, 2, 2, 2, 2, 1, 1, 2, 0, 3, 1, 3, 3, 1, 2, 2, 0, 3, 2, 3, 0, 1, 2, 0, 3, 1, 1, 3, 2, 2, 2, 3, 0, 3, 1, 3, 2, 1, 0, 1, 3, 0, 0, 2, 2, 0, 2, 3, 1, 2, 0, 1, 3, 3, 0, 0, 1, 1, 0, 0, 3, 1, 2, 3, 1, 2, 1, 1, 1, 2, 3, 0, 3, 3, 2, 0, 0, 3, 0, 3, 1, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 1, 2, 0, 0, 2, 2, 0, 3, 2, 2, 0, 0, 2, 0, 2, 2, 3, 2, 0, 2, 3, 2, 3, 1, 3, 0, 0, 2, 3, 1, 1, 1, 3, 0, 3, 3, 1, 2, 0, 2, 3, 2, 1, 2, 3, 1, 3, 2, 0, 2, 0, 3, 2, 3, 1, 2, 0, 3, 0, 1, 2, 2, 2, 3, 2, 2, 2, 0, 2, 2, 3, 1, 1, 0, 1, 2, 1, 2, 1, 3, 1, 2, 0, 1, 3, 2, 0, 0, 3, 2, 3, 0, 1, 0, 0, 2, 0, 3, 0, 3, 3, 0, 0, 1, 2, 1, 3, 1, 1, 0, 0, 0, 3, 2, 2, 3, 1, 1, 3, 0, 1, 3, 1, 2, 3, 1, 0, 2, 1, 3, 3, 3, 2, 1, 2, 1, 2, 0, 2, 3, 1, 0, 1, 0, 